# Loosely-coupled loop closure


In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
import cv2
from gtsam.symbol_shorthand import X
import matplotlib.pyplot as plt
import os

from lac.slam.visual_odometry import StereoVisualOdometry
from lac.utils.plotting import plot_poses, plot_surface, plot_3d_points
from lac.utils.visualization import image_grid
from lac.slam.loop_closure import estimate_loop_closure_pose
from lac.util import rotations_rmse_from_poses, positions_rmse_from_poses
from lac.util import load_data, load_stereo_images, load_side_images
from lac.params import LAC_BASE_PATH

%load_ext autoreload
%autoreload 2

In [ ]:
data_path = "/home/shared/data_raw/LAC/runs/full_spiral_map1_preset0_recovery_agent"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

fig = plot_poses(poses[:8000], no_axes=True, color="black", name="Ground truth")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

# Run VO to get odometry


In [ ]:
# Load the data logs
data_path = "/home/shared/data_raw/LAC/runs/full_spiral_map1_preset0_recovery_agent"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)
left_path = Path(data_path) / "FrontLeft"
right_path = Path(data_path) / "FrontRight"
side_path = Path(data_path) / "Right"

In [ ]:
svo = StereoVisualOdometry(cam_config)
svo_poses = []
pose_deltas = []

START_FRAME = 240
END_FRAME = 8000

print("Running VO...")
progress_bar = tqdm(range(START_FRAME, END_FRAME, 2), dynamic_ncols=True)

for frame in progress_bar:
    progress_bar.set_description(f"Processing Frame: {frame}")

    img_name = f"{frame:06}.png"
    left_img = cv2.imread(str(left_path / img_name), cv2.IMREAD_GRAYSCALE)
    right_img = cv2.imread(str(right_path / img_name), cv2.IMREAD_GRAYSCALE)

    if frame == START_FRAME:
        svo.initialize(poses[frame], left_img, right_img)
        svo_poses.append(poses[frame])
        continue

    svo.track(left_img, right_img)
    svo_poses.append(svo.rover_pose)
    pose_deltas.append(svo.pose_delta)

In [ ]:
fig = plot_poses(poses[START_FRAME:END_FRAME], no_axes=True, color="black", name="Ground truth")
fig = plot_poses(svo_poses, fig=fig, no_axes=True, color="orange", name="VO")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

# Detect loop closures

Position based


In [ ]:
frame1 = 800
frame2 = 3220

side_img1 = cv2.imread(str(side_path / f"{frame1:06}.png"), cv2.IMREAD_GRAYSCALE)
side_img2 = cv2.imread(str(side_path / f"{frame2:06}.png"), cv2.IMREAD_GRAYSCALE)

image_grid([side_img1, side_img2], rows=1, cols=2)

In [ ]:
# TODO: get a list of LC pairs
loop_closures = []

Plot the loop closures


In [ ]:
fig = plot_poses(poses[START_FRAME:END_FRAME], no_axes=True, color="black", name="Ground truth")
fig = plot_poses(svo_poses, fig=fig, no_axes=True, color="orange", name="VO")
fig.update_layout(height=900, width=1600, scene_aspectmode="data")
fig.show()

# Add loop closures

With gtsam EssentialMatrixFactor


In [ ]:
import gtsam
from gtsam.symbol_shorthand import X

In [ ]:
graph = gtsam.NonlinearFactorGraph()
values = gtsam.Values()

sigma_t = 0.005  # [m]
sigma_R = 0.00087  # [rad]
svo_pose_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([sigma_R, sigma_R, sigma_R, sigma_t, sigma_t, sigma_t]))

values.insert(X(0), gtsam.Pose3(initial_pose))
graph.add(gtsam.NonlinearEqualityPose3(X(0), gtsam.Pose3(initial_pose)))

i = 1
for frame in tqdm(np.arange(START_FRAME + 2, END_FRAME, 2)):
    values.insert(X(i), gtsam.Pose3(svo_poses[i]))
    graph.push_back(gtsam.BetweenFactorPose3(X(i - 1), X(i), gtsam.Pose3(pose_deltas[i - 1]), svo_pose_noise))
    i += 1